In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

instruments = ['hc', 'rb', 'i', 'j', 'jm', 'au', 'ag', 'v', 'ru', 'l', 'pp', 'bu', 'TA', 'FG', 'MA',
               'y', 'p', 'm', 'a', 'c', 'cs', 'jd', 'RM', 'CF', 'SR', 'OI']
start_date = 20180101
table = "AdjustedFuturesDaily"
engine = create_engine("sqlite:///../data/FuturesMarketData.db")
in_binds = ", ".join([f":sym{i}" for i in range(len(instruments))])
sql = text(f"""
    SELECT *, (ClosePrice * factor_multiply) as adjclose
    FROM {table}
    WHERE TradingDay >= :start
    AND Instrument IN ({in_binds})
""")

params = {"start": start_date} | {f"sym{i}": s for i, s in enumerate(instruments)}

with engine.begin() as conn:
    df = pd.read_sql(sql, conn, params=params)
df

In [ ]:
df['TradingDay'] = pd.to_datetime(df['TradingDay'].astype(str))
df = df.sort_values(['Instrument', 'TradingDay']).drop_duplicates(['Instrument', 'TradingDay'], keep='last')
cols_keep = ['TradingDay', 'Instrument', 'adjclose']
df = df[cols_keep]
df

In [ ]:
adj = (df
       .pivot_table(index='TradingDay',
                    columns='Instrument',
                    values='adjclose',
                    aggfunc='last')
       .sort_index()
       .sort_index(axis=1))

ax = adj.plot(title='Adjusted Futures – adjclose (all)')
ax.set_xlabel('Date'); ax.set_ylabel('Price')

In [ ]:
subset = ['rb','hc','i','au', 'CF', 'FG', 'jm']
adj[subset].dropna(how='all').plot(title='rb / hc / i / au – adjclose')
plt.xlabel('Date'); plt.ylabel('Price')
plt.show()